In [1]:
# 쥬피터노트북 셀 스타일 조절 
from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 100% !important; }</style>"))

# 경고 메세지 숨기기 
import warnings
warnings.filterwarnings(action='ignore')

C:\Users\ness9\AppData\Local\Temp\ipykernel_16504\829985655.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## 공공데이타포탈의 API 서비스 이용하기
- https://www.data.go.kr/
- 회원 가입후에 API서비스 신청
## 환경부 국립환경과학원_미세먼지(금속성분) 실시간 정보
조회날짜, 항목코드, 측정소코드, 시간구분을 기준으로 중금속 성분 측정 결과를 2시간 평균, 24시간 평균 측정 수치 자료로 제공하는 서비스

- https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15016368

- [활용신청] 클릭후 서비스키 정보 확인

- [마이페이지]-[오픈 API]-[개발 계정]

- 서비스키 예시
```
일반 인증키 (Encoding)
W7qHt8KLRZawPAd%2BiTUo1nt2EcSMgHwbxxMGhnKn3wfwLcwrVnHHmaVbPIi%2BgecqaouM%2ByjaLydMoXwiJm%2BMjQ%3D%3D
일반 인증키(Decoding)    
W7qHt8KLRZawPAd+iTUo1nt2EcSMgHwbxxMGhnKn3wfwLcwrVnHHmaVbPIi+gecqaouM+yjaLydMoXwiJm+MjQ==
```

In [2]:
# 0) 관련 모듈 임포트 
import json 
import pandas as pd
import requests
import urllib.request

In [3]:
# 1) API 서비스 요청 => response 객체 => json 텍스트 데이타
url = 'http://apis.data.go.kr/1480523/MetalMeasuringResultService/MetalService'
service_key = 'W7qHt8KLRZawPAd+iTUo1nt2EcSMgHwbxxMGhnKn3wfwLcwrVnHHmaVbPIi+gecqaouM+yjaLydMoXwiJm+MjQ=='
params ={'serviceKey' : service_key, 
         'pageNo' : '1', 
         'numOfRows' : '12', 
         'resultType' : 'JSON', 
         'date' : '20220602', 
         'stationcode' : '1', 
         'itemcode' : '90303', 
         'timecode' : 'RH02'}

response = requests.get(url, params=params)
print(response)
print(response.text)

<Response [200]>
{"MetalService":{"header":{"code":"00","message":"NORMAL SERVICE"},"item":[{"SDATE":"20220602000000","STATIONCODE":1,"ITEMCODE":"90303","TIMECODE":"RH02","VALUE":0.3307},{"SDATE":"20220602020000","STATIONCODE":1,"ITEMCODE":"90303","TIMECODE":"RH02","VALUE":0.1621},{"SDATE":"20220602040000","STATIONCODE":1,"ITEMCODE":"90303","TIMECODE":"RH02","VALUE":3.6916},{"SDATE":"20220602060000","STATIONCODE":1,"ITEMCODE":"90303","TIMECODE":"RH02","VALUE":8.8124},{"SDATE":"20220602080000","STATIONCODE":1,"ITEMCODE":"90303","TIMECODE":"RH02","VALUE":0.9511},{"SDATE":"20220602100000","STATIONCODE":1,"ITEMCODE":"90303","TIMECODE":"RH02","VALUE":1.9522},{"SDATE":"20220602120000","STATIONCODE":1,"ITEMCODE":"90303","TIMECODE":"RH02","VALUE":2.3849},{"SDATE":"20220602140000","STATIONCODE":1,"ITEMCODE":"90303","TIMECODE":"RH02","VALUE":2.3718},{"SDATE":"20220602160000","STATIONCODE":1,"ITEMCODE":"90303","TIMECODE":"RH02","VALUE":1.914},{"SDATE":"20220602180000","STATIONCODE":1,"ITEMCODE":"

In [ ]:
# station code 1~10 레코드 저장 
# 명세서 참고 stationcode 대기환경연구소 코드 대기환경연구소명 1 수도권 2 백령도 3 호남권 4 중부권 5 제주권 6 영남권 7 경기권 8 충청권 9 전북권 10 강원권

In [38]:
# 4) json 텍스트 => 딕셔너리 => 리스트 딕셔너리

# json 텍스트 => 딕셔너리
dust_dict = json.loads(response.text)
print(type(dust_dict))

# 딕셔너리 => 리스트 딕셔너리
dust_list = dust_dict['MetalService']['item']
print(type(dust_list))
dust_list[0]

<class 'dict'>
<class 'list'>


{'SDATE': '20220602000000',
 'STATIONCODE': 1,
 'ITEMCODE': '90303',
 'TIMECODE': 'RH02',
 'VALUE': 0.3307}

In [39]:
# 5) 리스트 딕셔너리 => 데이타프레임화
df_dust = pd.DataFrame(dust_dict['MetalService']['item'])
df_dust

,SDATE,STATIONCODE,ITEMCODE,TIMECODE,VALUE
0,20220602000000,1,90303,RH02,0.3307
1,20220602020000,1,90303,RH02,0.1621
2,20220602040000,1,90303,RH02,3.6916
3,20220602060000,1,90303,RH02,8.8124
4,20220602080000,1,90303,RH02,0.9511
5,20220602100000,1,90303,RH02,1.9522
6,20220602120000,1,90303,RH02,2.3849
7,20220602140000,1,90303,RH02,2.3718
8,20220602160000,1,90303,RH02,1.9140
9,20220602180000,1,90303,RH02,0.6874


## 모든 관측소의 데이타

- station code 1~10 레코드 저장
- station code 1~10 레코드 저장 
- 명세서 참고 stationcode 대기환경연구소 코드 대기환경연구소명 1 수도권 2 백령도 3 호남권 4 중부권 5 제주권 6 영남권 7 경기권 8 충청권 9 전북권 10 강원권

In [40]:
# 1) station 코드 => for 문
url = 'http://apis.data.go.kr/1480523/MetalMeasuringResultService/MetalService'
service_key = 'W7qHt8KLRZawPAd+iTUo1nt2EcSMgHwbxxMGhnKn3wfwLcwrVnHHmaVbPIi+gecqaouM+yjaLydMoXwiJm+MjQ=='
for i in range(1,11):
    params ={'serviceKey' : service_key, 
             'pageNo' : '1', 
             'numOfRows' : '12', 
             'resultType' : 'JSON', 
             'date' : '20220602', 
             'stationcode' : str(i), 
             'itemcode' : '90303', 
             'timecode' : 'RH02'}
    
    response = requests.get(url, params=params)
    # json 텍스트 => 딕셔너리 => 리스트 딕셔너리
    dust_dict = json.loads(response.text)
    
    # 리스트 추가
    dust_list += dust_dict['MetalService']['item']

In [41]:
len(dust_list)

120

In [44]:
df_dust = pd.DataFrame(dust_list)
df_dust

,SDATE,STATIONCODE,ITEMCODE,TIMECODE,VALUE
0,20220602000000,1,90303,RH02,0.3307
1,20220602020000,1,90303,RH02,0.1621
2,20220602040000,1,90303,RH02,3.6916
3,20220602060000,1,90303,RH02,8.8124
4,20220602080000,1,90303,RH02,0.9511
...,...,...,...,...,...
115,20220602140000,10,90303,RH02,0.0000
116,20220602160000,10,90303,RH02,0.1952
117,20220602180000,10,90303,RH02,0.0000
118,20220602200000,10,90303,RH02,0.5073


In [47]:
# 데이타프레임 데이타형 확인
df_dust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SDATE        120 non-null    object 
 1   STATIONCODE  120 non-null    int64  
 2   ITEMCODE     120 non-null    object 
 3   TIMECODE     120 non-null    object 
 4   VALUE        120 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.8+ KB


In [51]:
# station 코드값이 4번
df_dust[df_dust['STATIONCODE']==4].count()

SDATE          0
STATIONCODE    0
ITEMCODE       0
TIMECODE       0
VALUE          0
dtype: int64